# Pins and publishing

![](download.png)

## Pins

_The pins package publishes data, models, and other Python objects, making it easy to share them across projects and with your colleagues._

`pip install pins`

The pins package helps you publish data sets, models, and other R objects, making it easy to share them across projects and with your colleagues. The pin board can be on

- Local or temp folder
- RStudio Connect
- Amazon S3
- Azure storage
- Google Cloud

## When should I use pins?

-   When you're tired of emailing CSVs or models around
-   When you (or your colleagues) use R *and* Python for data tasks
-   When you want to save just a little bit of data in a visible place
-   When you want to version data or models in a lightweight way
-   When you share your model or data across multiple apps
-   When you don't want to redeploy your app every time the supporting data is updated

## How do I use pins?

Pins includes 2 major concepts: a board and a pinned object.

In [1]:
from pins import board_temp
from pins.data import mtcars

tmpboard = board_temp()
tmpboard

In [2]:
tmpboard.pin_write(mtcars.head(), "mtcars", type="csv")

Writing pin:
Name: 'mtcars'
Version: 20230710T164815Z-120a5


Meta(title='mtcars: a pinned 5 x 11 DataFrame', description=None, created='20230710T164815Z', pin_hash='120a54f7e0818041', file='mtcars.csv', file_size=249, type='csv', api_version=1, version=Version(created=datetime.datetime(2023, 7, 10, 16, 48, 15, 990642), hash='120a54f7e0818041'), tags=None, name='mtcars', user={}, local={})

In [3]:
tmpboard.pin_read("mtcars")

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


But, you can make pins more powerful with metadata.

In [4]:
tmpboard.pin_meta("mtcars")

Meta(title='mtcars: a pinned 5 x 11 DataFrame', description=None, created='20230710T164815Z', pin_hash='120a54f7e0818041', file='mtcars.csv', file_size=249, type='csv', api_version=1, version=Version(created=datetime.datetime(2023, 7, 10, 16, 48, 15), hash='120a5'), tags=None, name='mtcars', user={}, local={})

In [5]:
tmpboard.pin_write(
    mtcars,
    name="mtcars2",
    type="csv",
    description = "Data extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973–74 models).",
    metadata = {
        "source": "Henderson and Velleman (1981), Building multiple regression models interactively. Biometrics, 37, 391–411."
    }
)

Writing pin:
Name: 'mtcars2'
Version: 20230710T164824Z-3b134


Meta(title='mtcars2: a pinned 32 x 11 DataFrame', description='Data extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973–74 models).', created='20230710T164824Z', pin_hash='3b134bae183b50c9', file='mtcars2.csv', file_size=1333, type='csv', api_version=1, version=Version(created=datetime.datetime(2023, 7, 10, 16, 48, 24, 131030), hash='3b134bae183b50c9'), tags=None, name='mtcars2', user={'source': 'Henderson and Velleman (1981), Building multiple regression models interactively. Biometrics, 37, 391–411.'}, local={})

Pins can also be used to version objects.

In [6]:
board2 = board_temp()
board2.pin_write([1,2,3,4,5], name = "x", type = "json")
board2.pin_write([1,2,3], name = "x", type = "json")
board2.pin_write([1,2], name = "x", type = "json")
board2.pin_versions("x")

Writing pin:
Name: 'x'
Version: 20230710T164243Z-2bc5d
Writing pin:
Name: 'x'
Version: 20230710T164243Z-c24c0
Writing pin:
Name: 'x'
Version: 20230710T164243Z-91d9a


,created,hash,version
0,2023-07-10 16:42:43,2bc5d,20230710T164243Z-2bc5d
1,2023-07-10 16:42:43,91d9a,20230710T164243Z-91d9a
2,2023-07-10 16:42:43,c24c0,20230710T164243Z-c24c0


In [9]:
board2.pin_read('x', version='___')

[1, 2, 3]

You can bring pins to many platforms with different boards.

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("CO_API_KEY")
rsc_url = os.getenv("CO_RSC_URL")

In [7]:
from pins import board_connect

board = board_connect(server_url=rsc_url, api_key=api_key) # board_s3, board_gcs

board.pin_write(mtcars, "isabel.zimmerman/mtcars", type = "csv")

Writing pin:
Name: 'isabel.zimmerman/mtcars'
Version: 20230710T164835Z-3b134


Meta(title='mtcars: a pinned 32 x 11 DataFrame', description=None, created='20230710T164835Z', pin_hash='3b134bae183b50c9', file='mtcars.csv', file_size=1333, type='csv', api_version=1, version=VersionRaw(version='77094'), tags=None, name='isabel.zimmerman/mtcars', user={}, local={})

You can see this pin now in the Connect UI.

This allows for easy collaboration with R users!

In [8]:
board.pin_read('isabel.zimmerman/ten_numbers')

,x
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


`vetiver`, another package in the `pins` ecosystem, can help with model versioning tasks.

In [ ]:
from pins import board_temp
from vetiver import vetiver_pin_write

from vetiver.data import mtcars
from vetiver import VetiverModel
from sklearn import linear_model

X = mtcars.drop(columns="mpg") 
y = mtcars["mpg"]

car_mod = linear_model.LinearRegression().fit(X, y)
                 
v = VetiverModel(car_mod, model_name = "cars_mpg", 
                 prototype_data = X)

model_board = board_temp(allow_pickle_read = True)
vetiver_pin_write(model_board, v)

----

# Publishing on Connect

_This package provides a CLI (command-line interface) for interacting with and deploying to Posit Connect._

`pip install rsconnect-python`


## What can we all publish?

You are able to publish static content as well as API endpoints and web applications.

-   Jupyter Notebooks
-   Flask applications
-   FastAPI applications
-   Plotly Dash applications
-   Streamlit applications
-   Bokeh applications
-   Shiny applications
-   Voila applications

## How to publish content there?

This is generally done through the CLI tool, `rsconnect-python`.

You'll need

- a `.py`, `.qmd` or `.ipynb` file to deploy
- a server url and API key
- a `requirements.txt` file, optionally

In [ ]:
rsconnect deploy notebook \
    --server https://connect.example.org \
    --api-key my-api-key \
    my-notebook.ipynb

or `api`, `bokeh`, `dash`, `fastapi`, `html`, etc.. 

If you are always deploying to the same server, you can use the `add` command to store this information.

In [ ]:
rsconnect add \
    --api-key my-api-key \
    --server https://connect.example.org \
    --name myserver

Then you can reference the server by name in your command.

In [ ]:
rsconnect deploy notebook --name myserver my-notebook.ipynb

## My content is published, now what?

With published content, you can schedule runs, interact with your APIs, and pull down data.

## Resources

-   [pins for Python](https://rstudio.github.io/pins-python/intro.html), [GitHub](https://github.com/rstudio/pins-python)
-   [pins for R](https://github.com/rstudio/pins-r), [GitHub](https://pins.rstudio.com/)
-   [rsconnect-python](https://github.com/rstudio/rsconnect-python)
-   [Connect publishing](https://docs.posit.co/rsconnect-python/)
- [End to end workflow for model management, in R and Python](https://colorado.posit.co/rsc/connect/#/apps/0c1cad91-1e46-4eb9-9e15-713ce5c87566/access/6080)